# Cel

 * lepiej uczyć od warstwy FC czy razem z nią?
 * Najlepszy pooling
 * Najlepsza funkcja straty
 * Najlepszy optimizer

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

import cv2

import os

import keras
from keras.applications.resnet import ResNet50
from keras.layers import Dense
from keras.models import Sequential
from keras import optimizers
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
NUM_CLASSES = 2
CHANNELS = 3

IMAGE_RESIZE = 224
DENSE_LAYER_ACTIVATION = 'softmax'
LOSS_METRICS = ['accuracy']

NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

BATCH_SIZE_TRAINING = 32
BATCH_SIZE_VALIDATION = 32

BATCH_SIZE_TESTING = 1

In [ ]:
TOP = [False, True]
DATA_GEN = [False, True]
RESNET50_POOLING = ['avg', 'max']
OBJECTIVE_FUNCTION = ['mean_squared_error', 'categorical_crossentropy', 'sparse_categorical_crossentropy', 'binary_crossentropy']
OPTIMIZER = ['sgd', 'adam', 'adadelta']

In [ ]:
def exper1(top, pool, opt, loss, dataGen):
    out = []
    print(f'MODEL with:')
    print(f'\tinclude_top: {top}')
    print(f'\tpooling: {pool}')
    print(f'\toptimizer: {opt}')
    print(f'\tloss_func: {loss}')
    print(f'\tdata_gen: {dataGen}')
    
    # Model
    model = Sequential()
    model.add(ResNet50(include_top = top, pooling = pool, weights = 'imagenet'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
    model.layers[0].trainable = False
    model.summary()
    model.compile(optimizer = opt, loss = loss, metrics = LOSS_METRICS)
    
    # Data Gen
    image_size = IMAGE_RESIZE
    if dataGen:
        data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
    else:
        data_generator = ImageDataGenerator(
            shear_range=10,
            zoom_range=0.2,
            horizontal_flip=True,
            preprocessing_function=preprocess_input
        )
    
    # Data
    path = '../../MGU/Projekt/PS-Battles-master/' if os.name != 'nt' else '..\..\MGU\Data\PS-Battles-master'
    slash = '/' if os.name != 'nt' else '\\'
    train_generator = data_generator.flow_from_directory(
        directory=path + slash + 'test' + slash,
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='binary')

    validation_generator = data_generator.flow_from_directory(
        directory=path + slash + 'valid'+ slash,
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='binary') 
    
    # Callbacks
    cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
    cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')
    
    # Fit
    fit_history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator,
        validation_steps=STEPS_PER_EPOCH_VALIDATION,
        callbacks=[cb_checkpointer, cb_early_stopper]
    )
    model.load_weights("../working/best.hdf5")
    name = 'DogsCats_Top_' + str(top) + '_Pool_' + pool + '_Opt_' + opt + '_Loss_' + loss + '_dataGen_' + str(dataGen)
    model.save(name + '.h5')
    
    # Score v1
    scoreRes = model.evaluate_generator(
        generator=validation_generator
    )
    print(f'Accuracy = {scoreRes}')
    tmp = {}
    tmp[name] = scoreRes
    out.append(tmp)
    
    # Score v2
    plt.figure(1, figsize = (15,8)) 
    plt.subplot(221)  
    plt.plot(fit_history.history['accuracy'])  
    plt.plot(fit_history.history['val_accuracy'])  
    plt.title('model accuracy')  
    plt.ylabel('accuracy')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'valid']) 
    plt.subplot(222)  
    plt.plot(fit_history.history['loss'])  
    plt.plot(fit_history.history['val_loss'])  
    plt.title('model loss')  
    plt.ylabel('loss')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'valid']) 
    plt.show()
    
    # Show some predictions
    test_generator = data_generator.flow_from_directory(
        directory = path + slash + 'show' + slash,
        target_size = (image_size, image_size),
        batch_size = BATCH_SIZE_TESTING,
        class_mode = None,
        shuffle = False,
        seed = 123
    )
    test_generator.reset()
    pred = model.predict_generator(test_generator, steps = len(test_generator), verbose = 1)
    predicted_class_indices = np.argmax(pred, axis = 1)
    TEST_DIR = path + slash + 'show' + slash
    f, ax = plt.subplots(5, 5, figsize = (15, 15))
    for i in range(0,25):
        imgBGR = cv2.imread(TEST_DIR + test_generator.filenames[i])
        imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
        # a if condition else b
        predicted_class = "Dog" if predicted_class_indices[i] else "Cat"
        ax[i//5, i%5].imshow(imgRGB)
        ax[i//5, i%5].axis('off')
        ax[i//5, i%5].set_title("Predicted:{}".format(predicted_class))    
    plt.show()
    
    # End
    print('---KONIEC---')

In [16]:
# Waaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaat

for t in TOP:
    for d in DATA_GEN:
        for p in RESNET50_POOLING:
            for o in OPTIMIZER:
                for l in OBJECTIVE_FUNCTION:
                    exper1(top=t, pool=p, opt=o, loss=l, dataGen=d)

False False avg sgd mean_squared_error
False False avg sgd categorical_crossentropy
False False avg sgd sparse_categorical_crossentropy
False False avg sgd binary_crossentropy
False False avg adam mean_squared_error
False False avg adam categorical_crossentropy
False False avg adam sparse_categorical_crossentropy
False False avg adam binary_crossentropy
False False avg adadelta mean_squared_error
False False avg adadelta categorical_crossentropy
False False avg adadelta sparse_categorical_crossentropy
False False avg adadelta binary_crossentropy
False False max sgd mean_squared_error
False False max sgd categorical_crossentropy
False False max sgd sparse_categorical_crossentropy
False False max sgd binary_crossentropy
False False max adam mean_squared_error
False False max adam categorical_crossentropy
False False max adam sparse_categorical_crossentropy
False False max adam binary_crossentropy
False False max adadelta mean_squared_error
False False max adadelta categorical_crossentrop